## Paso 3: Técnica de Regresión Logística

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from itertools import combinations
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.exceptions import UndefinedMetricWarning
import warnings
import time

# Se cargan los datos en un DataFrame
df_datos = pd.read_csv("datos/credit_risk_dataset.csv")

columnas_importantes = ['person_age', 'person_income', 'loan_status', 'cb_person_cred_hist_length', 'loan_grade']

# Se crea una instancia de SimpleImputer con la estrategia de imputación como la media
imputer = SimpleImputer(strategy='mean')
df_datos[columnas_importantes] = imputer.fit_transform(df_datos[columnas_importantes])

# Crear una instancia de OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Ajustar y transformar la columna 'loan_grade' usando OrdinalEncoder
encoded_loan_grade = ordinal_encoder.fit_transform(df_datos[['loan_grade']])

# Crear un DataFrame con la codificación ordinal de 'loan_grade'
encoded_loan_grade = pd.DataFrame(encoded_loan_grade, columns=['loan_grade'])

# Eliminar la columna original 'loan_grade' y concatenar la nueva codificación ordinal
df_datos = pd.concat([df_datos.drop(columns=['loan_grade']), encoded_loan_grade], axis=1)

# Se crea una lista de todas las columnas
todas_columnas = ['person_age', 'person_income', 'cb_person_cred_hist_length', 'loan_grade']

# DataFrame para almacenar resultados
resultados = []

# Se genera una única vez el conjunto de datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_datos[todas_columnas], df_datos['loan_status'], test_size=0.2, random_state=100)

# Se generan todas las combinaciones posibles de columnas
for r in range(1, len(todas_columnas) + 1):
    combinaciones_columnas = combinations(todas_columnas, r)
    # Se itera sobre cada combinación de columnas
    for cols in combinaciones_columnas:
        start_time_training = time.time()
        # Se crea y entrena el modelo de clasificación de Regresión Logística
        clasificador_regresion_logistica = LogisticRegression()
        modelo_clasificador_regresion_logistica = clasificador_regresion_logistica.fit(X_train[list(cols)], y_train)
        end_time_training = time.time()

        start_time_testing = time.time()
        # Se predicen las salidas de datos de prueba
        y_test_predicho = modelo_clasificador_regresion_logistica.predict(X_test[list(cols)])

        # Se calculan las métricas para evaluar el modelo de clasificación
        mc = confusion_matrix(y_test, y_test_predicho)
        e = accuracy_score(y_test, y_test_predicho)

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
            p = precision_score(y_test, y_test_predicho)

        s = recall_score(y_test, y_test_predicho)
        f1 = f1_score(y_test, y_test_predicho)
        end_time_testing = time.time()

        # Se calculan los tiempos de ejecución
        training_time = end_time_training - start_time_training
        testing_time = end_time_testing - start_time_testing

        # Se almacenan los resultados en el DataFrame de resultados
        resultados.append([cols, mc, e, p, s, f1, training_time, testing_time])

# Se crea el DataFrame final
df_resultados = pd.DataFrame(resultados, columns=['Columnas', 'Matriz de Confusión', 'Exactitud', 'Precisión', 'Sensibilidad', 'F1 Score', 'Tiempo de Entrenamiento', 'Tiempo de Prueba'])

# Se ordena el DataFrame por Exactitud
df_resultados = df_resultados.sort_values(by='Exactitud', ascending=False)

# Se muestra el DataFrame final
print(df_resultados)


                                             Columnas  \
14  (person_age, person_income, cb_person_cred_his...   
11            (person_age, person_income, loan_grade)   
3                                       (loan_grade,)   
6                            (person_age, loan_grade)   
9            (cb_person_cred_hist_length, loan_grade)   
12  (person_age, cb_person_cred_hist_length, loan_...   
0                                       (person_age,)   
1                                    (person_income,)   
2                       (cb_person_cred_hist_length,)   
4                         (person_age, person_income)   
5            (person_age, cb_person_cred_hist_length)   
7         (person_income, cb_person_cred_hist_length)   
8                         (person_income, loan_grade)   
10  (person_age, person_income, cb_person_cred_his...   
13  (person_income, cb_person_cred_hist_length, lo...   

          Matriz de Confusión  Exactitud  Precisión  Sensibilidad  F1 Score  \
14  [[48